In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

# EDA

In [2]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [3]:
from data_cleaning import drop_unnecessary_columns , drop_rows_with_few_missing_values

df, sumaary = drop_unnecessary_columns(df) 




In [4]:
# dropping rows due to NAs 

df, summary = df_cleaned, summary = drop_rows_with_few_missing_values(df) 

summary

{'original_rows': 426880,
 'final_rows': 406053,
 'dropped_rows': 20827,
 'drop_percentage': 4.88,
 'columns_with_few_missing': ['year',
  'description',
  'fuel',
  'odometer',
  'lat',
  'long',
  'transmission',
  'model'],
 'missing_columns': [],
 'missing_counts_before': {'year': np.int64(1205),
  'description': np.int64(70),
  'fuel': np.int64(3013),
  'odometer': np.int64(4400),
  'lat': np.int64(6549),
  'long': np.int64(6549),
  'transmission': np.int64(2556),
  'model': np.int64(5277)},
 'total_missing_values': np.int64(29619)}

In [5]:
df.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
fuel                 0
odometer             0
title_status      6778
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
dtype: int64

# Filling missing values 

In [6]:
# filling missing values in title status with 'missing' 
from data_title_status import fill_missing_values 

df,summary = fill_missing_values(df) 
summary

{'column_name': 'title_status',
 'fill_value': 'missing',
 'missing_before': np.int64(6778),
 'missing_after': np.int64(0),
 'values_filled': np.int64(6778),
 'total_rows': 406053}

In [7]:
from data_transmission import fill_missing_values_transmission

df,summary = fill_missing_values_transmission(df) 
summary

{'column_name': 'transmission',
 'fill_value': 'automatic',
 'missing_before': np.int64(0),
 'missing_after': np.int64(0),
 'values_filled': np.int64(0),
 'total_rows': 406053}

In [8]:
#  standardization  and extracting info from model and description.
from model_extract import process_car_dataset

featured_eng=process_car_dataset(df)


Starting data extraction and cleaning...


/Users/dhruvpatel/Desktop/projects/DealPredection/DataCleaning/model_extract.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4 cylinders' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_clean.at[idx, field] = value


Data extraction and cleaning completed!


In [10]:
# null values reduced 
featured_eng.isnull().sum() - df_cleaned.isnull().sum()

cylinders           NaN
description         0.0
drive          -71030.0
fuel                0.0
id                  0.0
lat                 0.0
long                0.0
manufacturer   -12111.0
model               0.0
odometer            0.0
paint_color         0.0
price               0.0
region              0.0
region_url          0.0
state               0.0
title_status    -6778.0
transmission        0.0
type           -40512.0
year                0.0
dtype: float64

In [15]:
# dropping the rows where manufacturer is null 

featured_eng = featured_eng[featured_eng['manufacturer'].notna()]



# drive column cleaning 

In [17]:
# standardasition 
from drive_cleaning import process_drive_column
featured_eng=process_drive_column(featured_eng, 'drive')


Starting drive column cleaning...
Drive column cleaning completed!
Updated distribution:
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64
DRIVE COLUMN CLEANING REPORT

BEFORE CLEANING:
------------------------------
drive
4wd                  182408
fwd                  104328
rwd                   59363
allwheeldrive          2883
frontwheeldrive        1285
rearwheeldrive          637
front wheel drive         1
Name: count, dtype: int64

AFTER CLEANING:
------------------------------
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64

CHANGES:
------------------------------
frontwheeldrive: 1285 → 0 (-1285)
4wd: 182408 → 185291 (+2883)
fwd: 104328 → 105614 (+1286)
rearwheeldrive: 637 → 0 (-637)
front wheel drive: 1 → 0 (-1)
allwheeldrive: 2883 → 0 (-2883)
rwd: 59363 → 60000 (+637)

VALIDATION:
------------------------------
Total rows: 402328
Null values: 51423
Unexpected values: []
Column is clean: True

Drive column cleaning complete

In [18]:
# filling null values from research : (na values)
from drive_cleaning import fill_missing_drive_from_reference


featured_eng = fill_missing_drive_from_reference(featured_eng,
                                                 reference_file='/Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv'
                                                 )

 

Loading reference data from: /Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv
Loaded 83 model-drive mappings from reference file
Found 51423 rows with missing drive values
✓ Filled drive for model 'focus' → 'fwd'
✓ Filled drive for model 'optima' → 'fwd'
✓ Filled drive for model 'versa' → 'fwd'
✓ Filled drive for model 'monte carlo' → 'fwd'
✓ Filled drive for model 'accord ex-l' → 'fwd'
✓ Filled drive for model 'jetta' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'mustang' → 'rwd'
✓ Filled drive for model 'impala lt' → 'fwd'
✓ Filled drive for model 'avalon' → 'fwd'
✓ Filled drive for model 'cooper s' → 'fwd'
✓ Filled drive for model 'accord' → 'fwd'
✓ Filled drive for model 'fusion se' → 'fwd'
✓ Filled drive for model 'lesabre' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'sonata' → 

In [20]:
featured_eng['drive'].value_counts()



drive
4wd    185291
fwd    116757
rwd     61663
Name: count, dtype: int64

# Cleaning Model 

Clean in stages:
* Stage 1: Remove obvious junk (numbers, too short, too long)
* Stage 2: Extract core model from complex strings
* Stage 3: Standardize spelling and format
* Stage 4: Apply manufacturer-specific rules

In [ ]:
# Stage 1 : Remove obvious junk (numbers, too short, too long)

from model_cleaning import remove_numerical_models

featured_eng=remove_numerical_models(featured_eng)

featured_eng['model'].value_counts().to_csv('model_counts.csv')






In [24]:
# stage 2 : 

In [ ]:
featured_eng['manufacturer'].unique()

In [26]:
# featured_eng.to_csv('cleaned_data.csv')


In [27]:
# import pandas as pd 
# featured_eng = pd.read_csv('cleaned_data.csv')

In [24]:
featured_eng['model'].value_counts()


model
f-150                        7528
silverado 1500               4772
1500                         4139
camry                        3054
silverado                    2948
                             ... 
f-150 crew cab limited          1
g-20                            1
Isuzu Medium Duty Cabover       1
edge limited fwd                1
96 Suburban                     1
Name: count, Length: 26290, dtype: int64

In [ ]:
df_cleaned['cylinders'].unique()

In [ ]:
from model_cleaning import clean_models_with_list
 
featured_eng_new = clean_models_with_list(featured_eng) 

In [ ]:
df_cleaned['cylinders'].unique()

In [ ]:
featured_eng_new['model'].value_counts()

In [23]:
featured_eng_new.to_csv('clean_v1.csv')

NameError: name 'featured_eng_new' is not defined

# Stage 2 :

In [ ]:
import pandas as pd 
clean_df =pd.read_csv('clean_v1.csv')
clean_df.head()

In [ ]:
clean_df.isnull().sum()

# Drive missing value impute 

In [22]:
clean_df['type'].value_counts()

NameError: name 'clean_df' is not defined

In [39]:
# cleaning and standardasition 
clean_df['type'] = clean_df['type'].replace({'mini van': 'minivan', 'mini-van': 'minivan'})




In [ ]:
# Method 1: Using crosstab with percentages
drive_type_breakdown = pd.crosstab(clean_df['type'], clean_df['drive'], normalize='index') * 100
print("Drive Type Distribution by Car Type (Percentages):")
print(drive_type_breakdown.round(2))

print("\n" + "="*60 + "\n")


# based on the data , lets fill the null values with condition.  
# Assumptions : 
# 1 ) if the vehicle type are SUV ,  offroad , pickup , truck, other or wagon then fill ->  drive as 4wd 
# 2) if vehicle type are hatchback, minivan, sedan, van then fill -> drive fwd  
# 3 if  vehicle  are bus,convertible,coupe then fill rwd  
# for other 


In [ ]:
# impute - > Define the conditions and corresponding drive types
# Condition 1: SUV, offroad, pickup, truck, other, wagon -> 4WD
condition_4wd = clean_df['type'].isin(['SUV', 'offroad', 'pickup', 'truck', 'other', 'wagon'])

# Condition 2: hatchback, minivan, sedan, van -> FWD
condition_fwd = clean_df['type'].isin(['hatchback', 'minivan', 'sedan', 'van'])

# Condition 3: bus, convertible, coupe -> RWD
condition_rwd = clean_df['type'].isin(['bus', 'convertible', 'coupe'])

# Apply the imputation only to missing values
# Method 1: Using loc with conditions
clean_df.loc[(clean_df['drive'].isnull()) & condition_4wd, 'drive'] = '4wd'
clean_df.loc[(clean_df['drive'].isnull()) & condition_fwd, 'drive'] = '4wd'
clean_df.loc[(clean_df['drive'].isnull()) & condition_rwd, 'drive'] = 'rwd'

clean_df['drive'].value_counts()






In [ ]:
clean_df[clean_df['drive'].isna()]

In [43]:
clean_df.to_csv('clean_v1.csv')

In [ ]:
import pandas as pd 
clean_df =pd.read_csv('clean_v1.csv')
clean_df.head()

In [ ]:
clean_df.isnull().sum()

In [46]:
# droppign columns where drive ad type both is null 
clean_df = clean_df.dropna(subset=['drive', 'type'], how='all')

In [ ]:
clean_df.isnull().sum()

In [ ]:
clean_df['type'].value_counts()

In [ ]:
clean_df[clean_df['type'].isna()]['model'].value_counts()

In [ ]:
# filling null values based on data present. 
# First, let's see what we're working with
print("Models with missing types:")
print(clean_df[clean_df['type'].isna()]['model'].value_counts())

# Create a mapping of model to most common type
model_type_mapping = clean_df.groupby('model')['type'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Fill missing values using this mapping
clean_df['type'] = clean_df['type'].fillna(clean_df['model'].map(model_type_mapping))

# Check if there are still any missing values
print(f"\nRemaining missing values in 'type': {clean_df['type'].isna().sum()}")


In [ ]:
clean_df.isnull().sum()

In [ ]:
clean_df['paint_color'].value_counts()

In [53]:
clean_df['model'].value_counts().to_csv('model_counts.csv') 

In [54]:
# clean_df.to_csv('clean_v1.csv')

# Stage 3 

In [ ]:
import pandas as pd 

df =pd.read_csv('clean_v1.csv') 
df.head(10)

In [ ]:
df.columns

In [ ]:
df =df.drop(['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'],axis=1)
df.head(10) 

In [ ]:
df['model'].value_counts()

In [ ]:
len(df)

In [ ]:
from model_cleaning import filter_by_value_counts

df = filter_by_value_counts(df, 'model', min_count=10)

In [ ]:
len(df)

In [ ]:
df['model'].value_counts() 

In [ ]:
# lets explore other columns 

df['region'].value_counts()

In [ ]:
df['state'].value_counts()

In [ ]:
sorted(dict(df['manufacturer'].value_counts()))

In [17]:
df['manufacturer'] = df['manufacturer'].replace({
    'land rover': 'land-rover',
    'rover': 'land-rover',
    'land rover': 'land-rover'  # in case there are duplicates
})

In [ ]:
df[df['manufacturer']== 'harley']

In [ ]:
def validate_manufacturers(df, manufacturer_column='manufacturer'):
    """
    Validate and filter DataFrame to keep only rows with approved manufacturers
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    manufacturer_column (str): Name of the manufacturer column
    
    Returns:
    pd.DataFrame: Filtered DataFrame with only valid manufacturers
    dict: Summary of validation results
    """
    
    # Valid manufacturers list
    valid_manufacturers = [
        'acura', 'alfa-romeo', 'am-general', 'amc', 'audi', 'bentley', 'bmw', 
        'buick', 'cadillac', 'chevrolet', 'chrysler', 'dodge', 'eagle', 'ferrari', 
        'fiat', 'ford', 'freightliner', 'geo', 'gmc', 'hino', 'honda', 'hyundai', 
        'infiniti', 'international', 'isuzu', 'jaguar', 'jeep', 'kaiser', 'kenworth', 
        'kia', 'lamborghini', 'land-rover', 'lexus', 'lincoln', 'lotus', 'maserati', 
        'mazda', 'mclaren', 'mercedes-benz', 'mercury', 'mg', 'mini', 'mitsubishi', 
        'nash', 'nissan', 'oldsmobile', 'packard', 'peterbilt', 'plymouth', 'polaris', 
        'pontiac', 'porsche', 'ram', 'rolls-royce', 'saab', 'saturn', 'smart', 
        'sterling', 'studebaker', 'subaru', 'suzuki', 'tesla', 'toyota', 'triumph', 
        'volkswagen', 'volvo', 'vpg', 'western-star', 'willys'
    ]
    
    # Store original info
    original_count = len(df)
    original_manufacturers = df[manufacturer_column].value_counts()
    
    # Find invalid manufacturers
    invalid_manufacturers = df[~df[manufacturer_column].isin(valid_manufacturers)][manufacturer_column].value_counts()
    
    # Filter DataFrame
    filtered_df = df[df[manufacturer_column].isin(valid_manufacturers)].copy()
    
    # Create summary
    validation_summary = {
        'original_rows': original_count,
        'filtered_rows': len(filtered_df),
        'dropped_rows': original_count - len(filtered_df),
        'drop_percentage': round((original_count - len(filtered_df)) / original_count * 100, 2),
        'valid_manufacturers_count': len(valid_manufacturers),
        'found_manufacturers_count': len(original_manufacturers),
        'invalid_manufacturers': dict(invalid_manufacturers)
    }
    
    return filtered_df, validation_summary

# Usage
filtered_df, summary = validate_manufacturers(df)

# Print summary
print(f"Original rows: {summary['original_rows']}")
print(f"Filtered rows: {summary['filtered_rows']}")
print(f"Dropped rows: {summary['dropped_rows']} ({summary['drop_percentage']}%)")
print(f"\nInvalid manufacturers found:")
for manufacturer, count in summary['invalid_manufacturers'].items():
    print(f"  - {manufacturer}: {count} rows")

In [ ]:
df.isnull().sum()

In [ ]:
df['paint_color'].value_counts()

In [ ]:
def fill_paint_color_nulls(df, paint_color_col='paint_color', manufacturer_col='manufacturer', state_col='state'):
    """
    Fill null values in paint_color column based on most common color 
    for each manufacturer-state combination
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    paint_color_col (str): Name of the paint color column
    manufacturer_col (str): Name of the manufacturer column  
    state_col (str): Name of the state column
    
    Returns:
    pd.DataFrame: DataFrame with filled paint_color values
    dict: Summary of filling operation
    """
    
    # Make a copy to avoid modifying original
    df_filled = df.copy()
    
    # Count nulls before filling
    nulls_before = df_filled[paint_color_col].isnull().sum()
    total_rows = len(df_filled)
    
    # Step 1: Fill based on manufacturer + state combination
    manufacturer_state_mode = df_filled.groupby([manufacturer_col, state_col])[paint_color_col].agg(
        lambda x: x.mode().iloc[0] if len(x.mode()) > 0 and not x.mode().empty else None
    ).to_dict()
    
    # Create mask for nulls
    null_mask = df_filled[paint_color_col].isnull()
    
    # Fill nulls with manufacturer-state mode
    for idx in df_filled[null_mask].index:
        manufacturer = df_filled.loc[idx, manufacturer_col]
        state = df_filled.loc[idx, state_col]
        
        if (manufacturer, state) in manufacturer_state_mode:
            mode_color = manufacturer_state_mode[(manufacturer, state)]
            if pd.notna(mode_color):
                df_filled.loc[idx, paint_color_col] = mode_color
    
    # Step 2: Fill remaining nulls with manufacturer-only mode
    still_null_mask = df_filled[paint_color_col].isnull()
    manufacturer_mode = df_filled.groupby(manufacturer_col)[paint_color_col].agg(
        lambda x: x.mode().iloc[0] if len(x.mode()) > 0 and not x.mode().empty else None
    ).to_dict()
    
    for idx in df_filled[still_null_mask].index:
        manufacturer = df_filled.loc[idx, manufacturer_col]
        
        if manufacturer in manufacturer_mode:
            mode_color = manufacturer_mode[manufacturer]
            if pd.notna(mode_color):
                df_filled.loc[idx, paint_color_col] = mode_color
    
    # Step 3: Fill any remaining nulls with overall most common color
    remaining_null_mask = df_filled[paint_color_col].isnull()
    if remaining_null_mask.sum() > 0:
        overall_mode = df_filled[paint_color_col].mode()
        if len(overall_mode) > 0:
            df_filled.loc[remaining_null_mask, paint_color_col] = overall_mode.iloc[0]
    
    # Count nulls after filling
    nulls_after = df_filled[paint_color_col].isnull().sum()
    filled_count = nulls_before - nulls_after
    
    # Create summary
    filling_summary = {
        'total_rows': total_rows,
        'nulls_before': nulls_before,
        'nulls_after': nulls_after,
        'filled_count': filled_count,
        'fill_percentage': round((filled_count / nulls_before * 100), 2) if nulls_before > 0 else 0,
        'manufacturer_state_combinations': len(manufacturer_state_mode),
        'successful_combinations': sum(1 for v in manufacturer_state_mode.values() if pd.notna(v))
    }
    
    return df_filled, filling_summary

# Usage
df, summary = fill_paint_color_nulls(df)

# Print summary
print(f"Total rows: {summary['total_rows']}")
print(f"Nulls before: {summary['nulls_before']}")
print(f"Nulls after: {summary['nulls_after']}")
print(f"Filled: {summary['filled_count']} ({summary['fill_percentage']}%)")
print(f"Manufacturer-State combinations: {summary['manufacturer_state_combinations']}")
print(f"Successful combinations: {summary['successful_combinations']}")

In [ ]:
df.isnull().sum()

In [ ]:
df['region'].unique() 

In [ ]:
# adding new region column ( replacing older one) to trim down to 9 regions according to the u.s censue data 
df['state'].unique()


In [ ]:
def add_census_divisions_abbrev(df, state_col='state', new_col='census_region'):
    """
    Add U.S. Census Bureau Regional Divisions based on state abbreviations
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    state_col (str): Name of the state column (with abbreviations)
    new_col (str): Name of the new census division column
    
    Returns:
    pd.DataFrame: DataFrame with new census division column
    """
    
    # U.S. Census Bureau Regional Divisions mapping (state abbreviations)
    census_divisions = {
        # New England
        'ct': 'New England',  # Connecticut
        'me': 'New England',  # Maine
        'ma': 'New England',  # Massachusetts
        'nh': 'New England',  # New Hampshire
        'ri': 'New England',  # Rhode Island
        'vt': 'New England',  # Vermont
        
        # Middle Atlantic
        'nj': 'Middle Atlantic',  # New Jersey
        'ny': 'Middle Atlantic',  # New York
        'pa': 'Middle Atlantic',  # Pennsylvania
        
        # East North Central
        'il': 'East North Central',  # Illinois
        'in': 'East North Central',  # Indiana
        'mi': 'East North Central',  # Michigan
        'oh': 'East North Central',  # Ohio
        'wi': 'East North Central',  # Wisconsin
        
        # West North Central
        'ia': 'West North Central',  # Iowa
        'ks': 'West North Central',  # Kansas
        'mn': 'West North Central',  # Minnesota
        'mo': 'West North Central',  # Missouri
        'ne': 'West North Central',  # Nebraska
        'nd': 'West North Central',  # North Dakota
        'sd': 'West North Central',  # South Dakota
        
        # South Atlantic
        'de': 'South Atlantic',  # Delaware
        'fl': 'South Atlantic',  # Florida
        'ga': 'South Atlantic',  # Georgia
        'md': 'South Atlantic',  # Maryland
        'nc': 'South Atlantic',  # North Carolina
        'sc': 'South Atlantic',  # South Carolina
        'va': 'South Atlantic',  # Virginia
        'wv': 'South Atlantic',  # West Virginia
        'dc': 'South Atlantic',  # Washington DC
        
        # East South Central
        'al': 'East South Central',  # Alabama
        'ky': 'East South Central',  # Kentucky
        'ms': 'East South Central',  # Mississippi
        'tn': 'East South Central',  # Tennessee
        
        # West South Central
        'ar': 'West South Central',  # Arkansas
        'la': 'West South Central',  # Louisiana
        'ok': 'West South Central',  # Oklahoma
        'tx': 'West South Central',  # Texas
        
        # Mountain
        'az': 'Mountain',  # Arizona
        'co': 'Mountain',  # Colorado
        'id': 'Mountain',  # Idaho
        'mt': 'Mountain',  # Montana
        'nv': 'Mountain',  # Nevada
        'nm': 'Mountain',  # New Mexico
        'ut': 'Mountain',  # Utah
        'wy': 'Mountain',  # Wyoming
        
        # Pacific
        'ak': 'Pacific',  # Alaska
        'ca': 'Pacific',  # California
        'hi': 'Pacific',  # Hawaii
        'or': 'Pacific',  # Oregon
        'wa': 'Pacific'   # Washington
    }
    
    # Create a copy to avoid modifying original
    df_with_divisions = df.copy()
    
    # Map state abbreviations to census divisions
    df_with_divisions[new_col] = df_with_divisions[state_col].map(census_divisions)
    
    # Check for unmapped states
    unmapped_states = df_with_divisions[df_with_divisions[new_col].isnull()][state_col].unique()
    
    # Summary
    mapping_summary = {
        'total_rows': len(df_with_divisions),
        'mapped_rows': df_with_divisions[new_col].notna().sum(),
        'unmapped_rows': df_with_divisions[new_col].isna().sum(),
        'unmapped_states': list(unmapped_states),
        'divisions_found': df_with_divisions[new_col].nunique()
    }
    
    return df_with_divisions, mapping_summary

# Usage
df, summary = add_census_divisions_abbrev(df)

# Print summary
print(f"Total rows: {summary['total_rows']}")
print(f"Mapped rows: {summary['mapped_rows']}")
print(f"Unmapped rows: {summary['unmapped_rows']}")
print(f"Census divisions found: {summary['divisions_found']}")

if summary['unmapped_states']:
    print(f"Unmapped states: {summary['unmapped_states']}")

# View the distribution
print("\nCensus Division Distribution:")
print(df['census_region'].value_counts()) 

In [ ]:
import matplotlib.pyplot as plt

# Basic histogram
plt.figure(figsize=(10, 6))
plt.hist(df['year'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Distribution of Car Years')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def validate_years(df, year_column='year', min_year=1990):
    """
    Validate and filter DataFrame to keep only rows with years >= min_year
    
    Parameters:
    df (pd.DataFrame): Input DataFrame
    year_column (str): Name of the year column
    min_year (int): Minimum year to keep (default: 1990)
    
    Returns:
    pd.DataFrame: Filtered DataFrame with only valid years
    dict: Summary of validation results
    """
    
    # Store original info
    original_count = len(df)
    original_year_range = (df[year_column].min(), df[year_column].max())
    
    # Find invalid years (older than min_year)
    invalid_years_mask = df[year_column] < min_year
    invalid_years = df[invalid_years_mask][year_column].value_counts().sort_index()
    
    # Also check for null values
    null_years = df[year_column].isnull().sum()
    
    # Filter DataFrame to keep only valid years
    filtered_df = df[df[year_column] >= min_year].copy()
    
    # Remove null values as well
    filtered_df = filtered_df[filtered_df[year_column].notna()].copy()
    
    # Calculate new year range
    if len(filtered_df) > 0:
        new_year_range = (filtered_df[year_column].min(), filtered_df[year_column].max())
    else:
        new_year_range = (None, None)
    
    # Create summary
    validation_summary = {
        'original_rows': original_count,
        'filtered_rows': len(filtered_df),
        'dropped_rows': original_count - len(filtered_df),
        'drop_percentage': round((original_count - len(filtered_df)) / original_count * 100, 2),
        'min_year_threshold': min_year,
        'original_year_range': original_year_range,
        'new_year_range': new_year_range,
        'null_years': null_years,
        'invalid_years_count': len(invalid_years),
        'invalid_years_breakdown': dict(invalid_years)
    }
    
    return filtered_df, validation_summary

# Usage
df, summary = validate_years(df, year_column='year', min_year=1990)

# Print summary
print(f"Original rows: {summary['original_rows']}")
print(f"Filtered rows: {summary['filtered_rows']}")
print(f"Dropped rows: {summary['dropped_rows']} ({summary['drop_percentage']}%)")
print(f"Original year range: {summary['original_year_range'][0]} - {summary['original_year_range'][1]}")
print(f"New year range: {summary['new_year_range'][0]} - {summary['new_year_range'][1]}")
print(f"Null years: {summary['null_years']}")

if summary['invalid_years_breakdown']:
    print(f"\nInvalid years found (< {summary['min_year_threshold']}):")
    for year, count in summary['invalid_years_breakdown'].items():
        print(f"  - {year}: {count} cars")

In [ ]:
type(df['year'][0])

In [ ]:
# Check price statistics
print(df['price'].describe())
print(f"Price range: ${df['price'].min():,.0f} - ${df['price'].max():,.0f}")

# Check for extreme outliers
print(f"99th percentile: ${df['price'].quantile(0.99):,.0f}")
print(f"95th percentile: ${df['price'].quantile(0.95):,.0f}")

In [ ]:
def clean_price_data(df, price_col='price'):
    """Clean price data by removing invalid and extreme outliers"""
    
    original_count = len(df)
    
    # Remove invalid prices
    df_clean = df[df[price_col] > 0].copy()  # Remove $0 prices
    
    # Set reasonable price limits for used cars
    min_price = 500      # Minimum reasonable car price
    max_price = 100000   # Maximum reasonable used car price
    
    df_clean = df_clean[(df_clean[price_col] >= min_price) & 
                        (df_clean[price_col] <= max_price)]
    
    # Or use percentile-based filtering (more conservative)
    # q99 = df_clean[price_col].quantile(0.99)
    # df_clean = df_clean[df_clean[price_col] <= q99]
    
    dropped_count = original_count - len(df_clean)
    
    print(f"Original rows: {original_count:,}")
    print(f"Cleaned rows: {len(df_clean):,}")
    print(f"Dropped rows: {dropped_count:,} ({dropped_count/original_count*100:.1f}%)")
    
    # New statistics
    print(f"\nCleaned price range: ${df_clean[price_col].min():,.0f} - ${df_clean[price_col].max():,.0f}")
    print(f"New mean: ${df_clean[price_col].mean():,.0f}")
    print(f"New std: ${df_clean[price_col].std():,.0f}")
    
    return df_clean

# Clean your data
df = clean_price_data(df)

In [ ]:
import matplotlib.pyplot as plt

# Basic histogram
plt.figure(figsize=(10, 6))
plt.hist(df['price'], bins=30000, edgecolor='black', alpha=0.7)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Distribution of Car Price')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(df['price'], vert=False)
plt.xlabel('Price ($)')
plt.title('Price Distribution - Box Plot')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
df['transmission'].value_counts()

In [ ]:

df[df['fuel']=='other']

In [ ]:
# transmission column 

from data_transmission import convert_transmission_to_automatic , validate_transmission_values 

df, summary = convert_transmission_to_automatic(df)

# Print summary
print(f"Total rows: {summary['total_rows']}")
print(f"Original unique values: {summary['original_unique_values']}")
print(f"New unique values: {summary['new_unique_values']}")
print(f"Converted to automatic: {summary['converted_to_automatic']}")

print("\nOriginal value counts:")
for value, count in summary['original_value_counts'].items():
    print(f"  - {value}: {count}")

print("\nNew value counts:")
for value, count in summary['new_value_counts'].items():
    print(f"  - {value}: {count}") 


# Usage
df, validation_summary = validate_transmission_values(df)

# Print validation results
print(f"Validation Results:")
print(f"Total rows: {validation_summary['total_rows']}")
print(f"Valid rows: {validation_summary['valid_rows']}")
print(f"Dropped rows: {validation_summary['dropped_rows']} ({validation_summary['drop_percentage']}%)")
print(f"Validation passed: {validation_summary['validation_passed']}")

if validation_summary['invalid_values']:
    print(f"\nInvalid values found:")
    for value, count in validation_summary['invalid_values'].items():
        print(f"  - '{value}': {count} rows")

if validation_summary['null_values'] > 0:
    print(f"\nNull values: {validation_summary['null_values']}")

print(f"\nFinal value counts:")
for value, count in df['transmission'].value_counts().items():
    print(f"  - {value}: {count}")


In [ ]:
from data_fuel import validate_fuel_values, convert_fuel_to_gas

# Usage
df, summary = convert_transmission_to_automatic(df)

# Print summary
print(f"Total rows: {summary['total_rows']}")
print(f"Original unique values: {summary['original_unique_values']}")
print(f"New unique values: {summary['new_unique_values']}")
print(f"Converted to automatic: {summary['converted_to_automatic']}")

print("\nOriginal value counts:")
for value, count in summary['original_value_counts'].items():
    print(f"  - {value}: {count}")

print("\nNew value counts:")
for value, count in summary['new_value_counts'].items():
    print(f"  - {value}: {count}")

# Usage
df, validation_summary = validate_transmission_values(df)

# Print validation results
print(f"Validation Results:")
print(f"Total rows: {validation_summary['total_rows']}")
print(f"Valid rows: {validation_summary['valid_rows']}")
print(f"Dropped rows: {validation_summary['dropped_rows']} ({validation_summary['drop_percentage']}%)")
print(f"Validation passed: {validation_summary['validation_passed']}")

if validation_summary['invalid_values']:
    print(f"\nInvalid values found:")
    for value, count in validation_summary['invalid_values'].items():
        print(f"  - '{value}': {count} rows")

if validation_summary['null_values'] > 0:
    print(f"\nNull values: {validation_summary['null_values']}")

print(f"\nFinal value counts:")
for value, count in df['transmission'].value_counts().items():
    print(f"  - {value}: {count}")

In [ ]:
df.columns

In [ ]:


plt.figure(figsize=[10,6])
plt.hist(df['odometer'],bins=1000)
plt.xlabel('odometer')
plt.ylabel('Frequency')
plt.title('Distribution of odometer')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Import the functions
from data_odometer import process_odometer_column, preview_odometer_cleaning

# Preview what will be cleaned
preview_odometer_cleaning(df, 'odometer')

# Clean the data (recommended approach)
df, summary = process_odometer_column(df, 'odometer')

# Check results
print(f"Removed {summary['total_removed']} rows ({summary['total_removal_percentage']:.1f}%)")

In [ ]:


plt.figure(figsize=[10,6])
plt.hist(df['odometer'],bins=1000)
plt.xlabel('odometer')
plt.ylabel('Frequency')
plt.title('Distribution of odometer')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
df['title_status'].value_counts()

In [95]:
df.to_csv('clean_v2.csv')

In [1]:
import pandas as pd 

df = pd.read_csv('clean_v2.csv')

In [ ]:
df['title_status'].value_counts()

In [ ]:
from data_title_status import validate_title_status_values 

df, validation_summary = validate_title_status_values(df)

print("Title Status Validation Summary")
print("=" * 50)
print(f"Total rows: {validation_summary['total_rows']:,}")
print(f"Valid rows: {validation_summary['valid_rows']:,}")
print(f"Dropped rows: {validation_summary['dropped_rows']:,}")
print(f"Drop percentage: {validation_summary['drop_percentage']}%")
print(f"Validation passed: {'✅ Yes' if validation_summary['validation_passed'] else '❌ No'}")
    
print(f"\nValid values: {validation_summary['valid_values']}")

In [ ]:
df.columns

In [ ]:
df['type'].value_counts()

In [ ]:
from data_type import validate_type_values
df, summary = validate_type_values(df, standardize_case=True)


print(f"\n")

print(f"\nCleaned DataFrame (with standardized case):")
print(df)
print(f"\nCleaned value counts:")
print(df['type'].value_counts())
    

print(f"\n" + "="*60)
print("Example WITHOUT case standardization:")
cleaned_df_no_std, summary_no_std = validate_type_values(df, standardize_case=True)
print(f"Rows kept with case standardization: {len(df)}")
print(f"Rows kept without case standardization: {len(cleaned_df_no_std)}")
print(f"Difference: {len(df) - len(cleaned_df_no_std)} rows")

In [ ]:
df.columns